In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

Function to plot history recordings data.

In [2]:
import matplotlib.pyplot as plt

def plot_history(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

Generator.

In [3]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=True):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-01-11 17:34:10.689683: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-11 17:34:10.689802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



### Parte ormai inutile

Train and test arrays from generator.

In [6]:
import itertools
import numpy as np
from sklearn.model_selection import train_test_split

train_gen = itertools.islice(polar_generator(batchsize=1, grid=(10,10)), 100000)
print(type(train_gen))

<class 'itertools.islice'>


Shape check.

In [7]:
train = list(train_gen)

Array conversion.

In [8]:
X_train = np.array([x for x, y in train])
y_train = np.array([y for x, y in train]) #for array in y for label in array])

Array shape check.

In [9]:
print(X_train.shape)
print(y_train.shape)

(100000, 2, 1)
(100000, 1, 100)


### Models

#### Concatenated model

In [10]:
# 93%
from keras.layers import Input, Dense, concatenate, LeakyReLU, Dropout

theta = Input(shape=(1,))
rho = Input(shape=(1,))

x = concatenate([theta, rho])
#x = Dense(10, activation="relu")(x)
#x = Dense(20, activation=LeakyReLU(alpha=0.01))(x)
x = Dense(50, activation="relu")(x)
x = Dense(50, activation="tanh")(x)
#x = Dropout(0.1)(x)
#x = Dense(20, activation="tanh")(x)
#x = Dense(10, activation="tanh")(x)
#x = Dense(2, activation=LeakyReLU(alpha=0.1))(x)
out = Dense(100, activation='softmax')(x)

# Create model
model = Model(inputs=[theta, rho], outputs=out)
# Compile
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 2)            0           ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 50)           150         ['concatenate_1[0][0]']      

#### CNN Models

Good model!

In [130]:
# Good model! -> Without the MaxPooling Layer, it has reached a 93.17% accuracy on test
# Trained over 44/50 epochs without callback for LRAdjustments
# Batchsize: 64 // Steps per epoch: 10000

# Try to consider to add some dense layers 
# and to change the activation function of the last layer

# To improve the accuracy of the model, I could try to:
# - add more layers
# - change the activation function of the last layer
# - change the optimizer
# - change the loss function
# - change the learning rate
# - change the batch size
# - change the number of epochs

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

########################
model_name = "CNN_single_layer"
########################

optimizer = Adam(lr=0.01)

theta = Input(shape=(1,))
rho = Input(shape=(1,))

x = concatenate([theta, rho])
# Reshaping (theta, rho) to a 2D grid
x = Reshape((1,1,2))(x)
# Convolutional layer with 32 filters
x = Conv2D(16, (1, 1), activation='tanh')(x)
# Convolutional layer with 32 filters
x = Conv2D(32, (1, 1), activation='relu')(x)
# Max pooling layer
#x = MaxPooling2D((1, 1))(x) # I've tried to remove this layer and the performances do not seem to be affected by the removal
# Flatten the output
x = Flatten()(x)
# Fully connected layer with 100 neurons
x = Dense(100, activation='softmax')(x)


# Create model
model = Model(inputs=[theta, rho], outputs=x)

# Compile model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()


Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_107 (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 input_108 (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 concatenate_53 (Concatenate)   (None, 2)            0           ['input_107[0][0]',              
                                                                  'input_108[0][0]']              
                                                                                                  
 reshape_41 (Reshape)           (None, 1, 1, 2)      0           ['concatenate_53[0][0]']  

/Users/lorenzo/miniconda3/envs/MicroRacer_new/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [5]:
# Recreate the exact same model, including its weights and the optimizer
CNN_singleLayer_model = tf.keras.models.load_model('../Saved Models/CNN_single_layer.h5')

2023-01-11 17:35:01.727859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-11 17:35:01.727897: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


----------------

In [85]:
from keras.layers import Input, Conv2D,Conv1D,  MaxPooling2D, Flatten, Dense, Dropout

#Batch 64, it goes around 92%

########################
model_name = "CNN_DualDense_layer"
########################

theta = Input(shape=(1,))
rho = Input(shape=(1,))

x = concatenate([theta, rho])
# Reshaping (theta, rho) to a 2D grid
x = Reshape((1,1,2))(x)
# Convolutional layer with 32 filters
x = Conv2D(64, (1, 1), activation='relu')(x)
# Max pooling layer
#x = MaxPooling2D((1, 1))(x) # I've tried to remove this layer and the performances do not seem to be affected by the removal
x = Dense(32, activation='tanh')(x)
# Flatten the output
x = Flatten()(x)
# Fully connected layer with 100 neurons
x = Dense(100, activation='softmax')(x)


# Create model
model = Model(inputs=[theta, rho], outputs=x)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_77 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_78 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 concatenate_38 (Concatenate)   (None, 2)            0           ['input_77[0][0]',               
                                                                  'input_78[0][0]']               
                                                                                                  
 reshape_33 (Reshape)           (None, 1, 1, 2)      0           ['concatenate_38[0][0]']  

In [15]:
########################
model_name = "TripleDense_layer"
########################

# Define input tensors
theta = Input(shape=(1,))
rho = Input(shape=(1,))

# Concatenate the two inputs
x = concatenate([theta, rho])

# Add fully connected layers
x = Dense(10, activation='relu')(x)
#x = Dense(20, activation='tanh')(x)# before there was the tanh
#x = Dense(16, activation='tanh')(x)

# Add final output layer
x = Dense(100, activation='softmax')(x)

# Create model
model = Model(inputs=[theta, rho], outputs=x)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 2)            0           ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 10)           30          ['concatenate_1[0][0]']    

### Training the model

In [51]:
#tf.config.run_functions_eagerly(True)
#tf.data.experimental.enable_debug_mode()

Callbacks definitions.

In [126]:
from keras import callbacks

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="auto", patience = 8, 
                                        restore_best_weights = True)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                        patience=4, min_lr=0.00001)

checkpoint_path = "Best_models/" + model_name + ".h5"
checkpoint = callbacks.ModelCheckpoint(checkpoint_path,
                                       monitor='val_loss', verbose=1,
                                       save_best_only=True, mode='auto')


Effective train.

In [127]:
batchsize = 64
epochs = 200
steps_per_epoch = 54000//batchsize
validation_steps = 6000//batchsize

In [131]:
# Remember, if done the fit with an instance of the generator, to rebuild the instance,
# or it will produce an error interrupting iterations

"""
Another way could've been to instantiate the fit directly from the generator

history = model.fit(polar_generator(batchsize=32, grid=(10,10)),
            validation_data=polar_generator(batchsize=32, grid=(10,10)), 
            steps_per_epoch=1000, 
            validation_steps=100, 
            epochs=1000)
        
"""
with tf.device('/device:GPU:0'):
    history = model.fit(polar_generator(batchsize=batchsize, grid=(10,10)),
                        validation_data=polar_generator(batchsize=batchsize, grid=(10,10)), 
                        steps_per_epoch=steps_per_epoch, 
                        validation_steps=validation_steps, 
                        epochs=epochs,
                        shuffle=True, 
                        callbacks =[checkpoint, reduce_lr, earlystopping])

Epoch 1/200


2023-01-11 16:59:28.908850: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


839/843 [============================>.] - ETA: 0s - loss: 1.5289 - categorical_accuracy: 0.4972

2023-01-11 16:59:38.429284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss did not improve from 0.50275
843/843 [==============================] - 11s 12ms/step - loss: 1.5254 - categorical_accuracy: 0.4980 - val_loss: 0.8216 - val_categorical_accuracy: 0.6784 - lr: 0.0100
Epoch 2/200
841/843 [============================>.] - ETA: 0s - loss: 0.6619 - categorical_accuracy: 0.7362
Epoch 2: val_loss did not improve from 0.50275
843/843 [==============================] - 10s 11ms/step - loss: 0.6617 - categorical_accuracy: 0.7363 - val_loss: 0.6330 - val_categorical_accuracy: 0.7485 - lr: 0.0100
Epoch 3/200
839/843 [============================>.] - ETA: 0s - loss: 0.5315 - categorical_accuracy: 0.7848
Epoch 3: val_loss improved from 0.50275 to 0.43254, saving model to Best_models/CNN_single_layer.h5
843/843 [==============================] - 10s 12ms/step - loss: 0.5309 - categorical_accuracy: 0.7850 - val_loss: 0.4325 - val_categorical_accuracy: 0.8269 - lr: 0.0100
Epoch 4/200
840/843 [============================>.] - ETA: 0s - loss: 0.4957

In [132]:
print(history.history["val_categorical_accuracy"])

[0.6784273982048035, 0.7484878897666931, 0.8269489407539368, 0.8358535170555115, 0.8244287371635437, 0.8518145084381104, 0.8341733813285828, 0.789818525314331, 0.8103158473968506, 0.8587029576301575, 0.8469421863555908, 0.8526545763015747, 0.8534946441650391, 0.8563507795333862, 0.9480846524238586, 0.961693525314331, 0.9457325339317322, 0.9514448642730713, 0.9502688050270081, 0.9501007795333862, 0.9527889490127563, 0.9502688050270081, 0.9492607712745667, 0.9502688050270081, 0.9526209831237793, 0.9479166865348816, 0.9472446441650391, 0.948924720287323, 0.9529569745063782, 0.9492607712745667, 0.9536290168762207, 0.9442204236984253, 0.9544690847396851, 0.9564852118492126, 0.9544690847396851, 0.9539650678634644, 0.9465726017951965, 0.9539650678634644, 0.9539650678634644, 0.9556451439857483, 0.9514448642730713, 0.9492607712745667, 0.9455645084381104, 0.9546371102333069, 0.9692540168762207, 0.9623655676841736, 0.9652217626571655, 0.9682459831237793, 0.9658938050270081, 0.9711021780967712, 0.

In [10]:
plot_history(history)

NameError: name 'history' is not defined

### Predictions and accuracy evaluation

#### Predictions

In [6]:
model = CNN_singleLayer_model

In [13]:
test_gen = polar_generator(batchsize=20000, grid=(10,10))
((theta,rho),maps) = next(test_gen)
pred = model.predict([theta,rho])
len(pred)

2023-01-11 18:26:47.029507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20000

In [8]:
from tqdm import tqdm
def compute_accuracy(model, generator, num_samples):
    test_gen = polar_generator(batchsize=1, grid=(10,10))
    acc = 0
    for i in tqdm(range(num_samples)):
        ((theta,rho),maps) = next(test_gen)
        pred = model.predict([theta,rho])
        if np.argmax(pred) == np.argmax(maps):
            #print(np.argmax(pred[i]) == np.argmax(maps[i]))
            acc += 1
    return 100*(acc/num_samples)

In [9]:
with tf.device('/device:GPU:0'):
    accuracy = compute_accuracy(model, polar_generator, 20000)

100%|██████████| 20000/20000 [09:18<00:00, 35.81it/s]


In [10]:
accuracy

96.875

#### Accuracy

------------

### Some test on final predictions

In [ ]:
y_real_test[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [ ]:
# np.argmax() retrieves the position inside the element that has the higher value
np.argmax(y_real_test[0])

89

In [38]:
y_pred = model.predict(real_test_gen)

StopIteration: 

In [ ]:
y_pred[0]

array([0.00972783, 0.01054782, 0.00985779, 0.0103849 , 0.00981623,
       0.01039358, 0.01036221, 0.0101087 , 0.00956898, 0.00992909,
       0.00981386, 0.009505  , 0.00955362, 0.01014952, 0.01063547,
       0.00978287, 0.01016095, 0.01024174, 0.00964302, 0.00983779,
       0.00975216, 0.00999128, 0.0099328 , 0.0100944 , 0.00948367,
       0.00998045, 0.01015159, 0.0100207 , 0.01023646, 0.0105612 ,
       0.01025754, 0.0104017 , 0.01010685, 0.00994946, 0.01034355,
       0.00971982, 0.00962096, 0.00976536, 0.0102027 , 0.01018209,
       0.00974328, 0.00999209, 0.00995262, 0.01000857, 0.01088253,
       0.01032031, 0.00969054, 0.0100836 , 0.00960697, 0.01030811,
       0.00982615, 0.01012846, 0.00957684, 0.00969281, 0.01018658,
       0.00969104, 0.01014712, 0.01000969, 0.0097687 , 0.01010991,
       0.00953308, 0.01018469, 0.01028776, 0.00962206, 0.0101076 ,
       0.0106229 , 0.01005223, 0.00952791, 0.01024099, 0.01041296,
       0.01017077, 0.00995169, 0.00987118, 0.00972568, 0.00946

In [ ]:
np.argmax(y_pred[0])

44